# Session 4: Final Challenge and Reflection

Immanuel Williams & Ciera Millard  
2025-07-16

[Download starter .qmd file](04_Extraction_FinalActivity.qmd)

# Session 4: Final Project Activity

In this capstone session, we will combine our API and web scraping
skills to complete two mini-projects.

Each project will involve writing a custom function to extract data and
then using that function to create a unique data visualization.

## Part 1. Goals & Objectives

1.  **Synthesize Skills:** Combine API requests and web scraping
    techniques in two separate, start-to-finish projects.
2.  **Develop Reusable Functions:** Write modular R functions that
    handle the logic for extracting and cleaning data, making your code
    more readable and reusable.
3.  **Master Data Cleaning:** Practice essential data cleaning skills
    required to transform raw, messy data from both APIs and HTML tables
    into a format ready for analysis.
4.  **Create Meaningful Visualizations:** Use `ggplot2` to create two
    distinct data visualizations—a geographic map and a sorted bar
    chart—to effectively communicate the findings from your extracted
    data.

## Part 2: Conceptual Demonstration through projects

### **Project 1: Mapping U.S. Cities with the Geocoding API**

Our first goal is to take a list of U.S. cities, find their exact
geographic coordinates using an API, and plot them on a map. For this,
we’ll use a new OpenWeatherMap endpoint: the **Geocoding API (v1.0)**.

#### **Step 1: Load Libraries**

We’ll begin by loading all the libraries we need for API calls, data
wrangling, and visualization.

**Filled Version (Answer)**

In [ ]:
library(httr2)      # For making API requests
library(rvest)      # For web scraping
library(purrr)      # For functional programming like map()
library(glue)       # For building strings (URLs)
library(dplyr)      # For data manipulation
library(ggplot2)    # For plotting

**Fill-in-the-Blank Version**

In [ ]:
## EMPTY VERSION
# library(____)       # For making API requests
# library(____)       # For web scraping
# library(____)       # For functional programming like map()
# library(____)       # For building strings (URLs)
# library(____)       # For data manipulation
# library(____)       # For plotting

**Scratch Version Instructions:** Start by loading all the R packages
required for this project using the `library()` function. You will need
**`httr2`** to make requests to the API, **`purrr`** for applying your
function across multiple cities, **`glue`** to easily construct the
request URL, **`dplyr`** for data manipulation steps like selecting
columns, and **`ggplot2`** to create the final map visualization.

In [ ]:
## Completely EMPTY VERSION
# TODO: Load all necessary libraries for this project
# # For making API requests
# # For web scraping
# # For functional programming like map()
# # For building strings (URLs)
# # For data manipulation
# # For plotting

------------------------------------------------------------------------

#### **Step 2: Create a Geocoding Function**

Next, we’ll write a function named `get_city_coords`. This function will
take a city name as input, send a request to the OpenWeather Geocoding
API, and return a clean data frame with the city’s name, latitude, and
longitude.

**Filled Version (Answer)**

In [ ]:
get_city_coords <- function(city_name) {
  # Construct the request URL using the Geocoding v1.0 endpoint
  req_url <- glue("http://api.openweathermap.org/geo/1.0/direct?q={URLencode(city_name)}&limit=1&appid={Sys.getenv('OPENWEATHER_API_KEY')}")
  
  # Perform the request
  response <- request(req_url) %>% 
    req_perform()
  
  # Check for success and process the JSON response
  if (resp_status(response) == 200) {
    resp_body_json(response) %>% 
      pluck(1) %>% # Pluck the first result from the list
      as_tibble() %>% 
      select(name, lat, lon)
  } else {
    warning(paste("Could not find coordinates for:", city_name))
    return(NULL)
  }
}

**Fill-in-the-Blank Version**

In [ ]:
## EMPTY VERSION
# get_city_coords <- function(city_name) {
#   # Construct the request URL
#   req_url <- glue("http://api.openweathermap.org/geo/1.0/direct?q={____(city_name)}&limit=1&appid={Sys.getenv('OPENWEATHER_API_KEY')}")
#   
#   # Perform the request
#   response <- request(____) %>% 
#     req_perform()
#   
#   # Check for success (status code 200)
#   if (resp_status(____) == ____) {
#     resp_body_json(____) %>% 
#       pluck(____) %>% # Pluck the first result
#       as_tibble() %>% 
#       select(name, lat, lon)
#   } else {
#     warning(paste("Could not find coordinates for:", city_name))
#     return(NULL)
#   }
# }

**Scratch Version Instructions:** Write a function called
`get_city_coords` that accepts a single argument, `city_name`. Inside
the function, use `glue()` to construct the full request URL for the
OpenWeather Geocoding API, making sure to use `URLencode()` on the
`city_name` to handle spaces and special characters. Perform the API
call using `httr2::request()` and `req_perform()`. Then, use an `if`
statement to check if the response status is 200 (OK). If it is, parse
the JSON body, `pluck()` the first result from the list, convert it to a
tibble, and `select()` only the `name`, `lat`, and `lon` columns to
return. If the status is not 200, issue a `warning()` with the city name
and return `NULL`.

In [ ]:
## Completely EMPTY VERSION
# TODO: Write a function named get_city_coords that takes a city_name,
# calls the Geocoding API, and returns a tibble with name, lat, and lon.

------------------------------------------------------------------------

#### **Step 3: Create a Plot of City Locations**

##### **Substep 3a: Get U.S. Map Data**

To create our map background, we use the `map_data()` function. This
function retrieves a data frame containing the geographic polygon
data—essentially a set of longitude and latitude coordinates—needed to
draw the outlines of all U.S. states.

Code snippet

In [ ]:
# Get U.S. map data
us_map <- map_data("state")

**How This Works & Which Library**

-   **Library**: The `map_data()` function comes from the **`ggplot2`**
    package. It is designed specifically to get map data in a format
    that works seamlessly with `ggplot()`.

-   **Process**:

    1.  When you call `map_data("state")`, `ggplot2` accesses a built-in
        dataset (from the **`maps`** package) that contains the outlines
        of the world, countries, and U.S. states.

    2.  It returns a data frame where each row represents a single point
        (vertex) on the border of a state.

    3.  Crucially, it includes a `group` column. This column tells
        `ggplot2` which points should be connected to form a single,
        continuous shape (a state polygon), preventing lines from being
        drawn incorrectly across the map.

Now, we’ll define a list of cities and use our new function with
`purrr::map_dfr` to get the coordinates for all of them. Finally, we’ll
use `ggplot2` to plot these locations on a map of the United States.

**Filled Version (Answer)**

In [ ]:
# A vector of cities to map
cities_to_map <- c("San Luis Obispo", "Chicago", "New York", "Atlanta", "Houston", "Des Moines")

# Apply our function to each city and combine results into one data frame
city_locations_df <- map_dfr(cities_to_map, get_city_coords)



# Create the plot
us_map %>% 
  ggplot() +
  geom_polygon(aes(x = long, y = lat, group = group), fill = "gray90", color = "white") +
  geom_point(data = city_locations_df, aes(x = lon, y = lat), color = "red", size = 3) +
  labs(
    title = "Locations of Selected U.S. Cities",
    x = "Longitude",
    y = "Latitude"
  ) +
  theme_minimal()

In [ ]:
## EMPTY VERSION
## A vector of cities to map
# cities_to_map <- c("San Luis Obispo", "Chicago", "New York", "Atlanta", "Houston", "Des Moines")
# 
## Apply our function to each city using map_dfr
# city_locations_df <- map_dfr(____, ____)
# 
## Get U.S. map data
# us_map <- map_data("____")
# 
## Create the plot
# ggplot() +
#   geom_polygon(data = ____, aes(x = long, y = lat, group = group), fill = "gray90", color = "white") +
#   geom_point(data = ____, aes(x = lon, y = lat), color = "red", size = 3) +
#   labs(
#     title = "Locations of Selected U.S. Cities",
#     x = "Longitude",
#     y = "Latitude"
#   ) +
#   theme_minimal()

**Scratch Version Instructions:** First, create a character vector named
`cities_to_map` containing the names of several U.S. cities you want to
plot. Next, use the `purrr::map_dfr()` function to apply your
`get_city_coords` function to each city in your vector, storing the
combined results in a data frame called `city_locations_df`. Then,
create the base map object by calling `map_data("state")`. Finally,
build a `ggplot` by first calling `geom_polygon()` to draw the states
from the map data, followed by `geom_point()` to plot the city
coordinates from your `city_locations_df`. Remember to set the
aesthetics (`aes`) correctly for each layer and add appropriate titles
and labels using `labs()`.

In [ ]:
## Completely EMPTY VERSION
# TODO: Create a vector of city names.
# TODO: Use map_dfr() and your get_city_coords function to get their locations.
# TODO: Create a ggplot map of the US and add points for each city.

------------------------------------------------------------------------

Of course. Here is the project completely revised to focus on the Box
Office Mojo data, following the same structured, multi-version format.

------------------------------------------------------------------------

### **Project 2: Scraping and Plotting Box Office Data**

For this project, our goal is to scrape a list of the top-grossing films
of all time directly from the **Box Office Mojo** website and create a
visualization of the highest earners.

------------------------------------------------------------------------

#### **Step 1: Scrape Top-Grossing Movies**

First, we will scrape the Box Office Mojo website to get a clean table
of the top lifetime grossing films.

**Filled Version (Answer)**

In [ ]:
library(rvest)
library(janitor)
library(tibble)
library(purrr)

# Define the URL for the Box Office Mojo chart
mojo_url <- "https://www.boxofficemojo.com/chart/top_lifetime_gross/"

# Read the HTML, find the table, and clean it up
top_movies_df <- read_html(mojo_url) %>% 
  html_table() %>% 
  pluck(1) %>% 
  clean_names() 

top_movies_df

**Fill-in-the-Blank Version**

In [ ]:
## EMPTY VERSION
# library(rvest)
# library(janitor)
# library(purrr)

## Define the URL for the Box Office Mojo chart
# mojo_url <- "https://www.boxofficemojo.com/chart/top_lifetime_gross/"
# 
## Read the HTML, find the table, and clean it up
# top_movies_df <- read_*(____) %>% 
#   html_*() %>% 
#   pluck(____) %>% # Which table on the page is it?
#   clean_names() 
# 
# print(top_movies_df)

**Scratch Version Instructions:** Begin with the `mojo_url` provided.
Create a data pipeline that first uses `rvest::read_html()` to download
the page’s content. Pipe this into `rvest::html_table()` to extract all
tables on the page into a list. Since you want the main table, use
`purrr::pluck(1)` to select the first item from that list. Finish the
pipeline by using `janitor::clean_names()` to standardize the messy
column headers and `tibble::as_tibble()` to ensure the final result is a
tibble named `top_movies_df`.

In [ ]:
## Completely EMPTY VERSION
## TODO: Scrape the table of top-grossing films from the mojo_url.
## TODO: Clean the data using janitor::clean_names() and convert it to a tibble.

------------------------------------------------------------------------

#### **Step 2: Create a Function to Plot Top Movies**

The data we scraped has the gross earnings stored as text (e.g.,
“\$2,923,706,026”). We need to write a function that takes this raw data
frame, cleans the `lifetime_gross` column so it can be treated as a
number, and then plots the top 10 movies in a bar chart.

**Filled Version (Answer)**

In [ ]:
## FILLED VERSION
library(ggplot2)
library(dplyr)
library(readr)
library(stringr)

plot_top_grossing_movies <- function(data) {
  # Clean the data: convert currency text to numbers
  plot_data <- data %>%
    mutate(
      lifetime_gross_numeric = parse_number(lifetime_gross)
    ) %>% 
    # Get the top 10 movies by gross
    slice_max(order_by = lifetime_gross_numeric, n = 10)

  # Create the bar chart
  ggplot(plot_data, aes(x = lifetime_gross_numeric, y = reorder(title, lifetime_gross_numeric))) +
    geom_col(fill = "#e76f51") +
    scale_x_continuous(labels = scales::label_dollar(scale = 1/1e9, suffix = "B")) + # Format x-axis in billions
    labs(
      title = "Top 10 Lifetime Grossing Films",
      subtitle = "Source: Box Office Mojo",
      x = "Lifetime Gross (in Billions USD)",
      y = "Film Title"
    ) +
    theme_minimal()
}

**Fill-in-the-Blank Version**

In [ ]:
##EMPTY VERSION
# library(ggplot2)
# library(dplyr)
# library(readr)
# library(stringr)
# plot_top_grossing_movies <- function(data) {
#   # Clean the data
#   plot_data <- data %>%
#     mutate(
#       lifetime_gross_numeric = parse_number(____) # Convert the currency column
#     ) %>% 
#     slice_max(order_by = ____, n = 10) # Get top 10 movies
# 
#   # Create the bar chart
#   ggplot(plot_data, aes(x = ____, y = reorder(title, ____))) +
#     geom_col(fill = "#e76f51") +
#     scale_x_continuous(labels = scales::label_dollar(scale = 1/1e9, suffix = "B")) +
#     labs(
#       title = "Top 10 Lifetime Grossing Films",
#       subtitle = "Source: Box Office Mojo",
#       x = "Lifetime Gross (in Billions USD)",
#       y = "Film Title"
#     ) +
#     theme_minimal()
# }

**Scratch Version Instructions:** Write a function named
`plot_top_grossing_movies` that takes one argument, `data`. Inside the
function, use `dplyr` to create a `plot_data` object. This pipeline
should start with the input `data`, then use `mutate()` and
`readr::parse_number()` to create a new numeric column from the
`lifetime_gross` text column. Next, use `slice_max()` to keep only the
top 10 movies based on this new numeric column. Finally, pipe this
`plot_data` into `ggplot()`. In your `aes()`, map the numeric gross to
the x-axis and use `reorder(title, lifetime_gross_numeric)` for the
y-axis to ensure the bars are sorted. Use `geom_col()` to create the bar
chart and `scale_x_continuous()` with `scales::label_dollar()` to format
the x-axis labels in billions.

In [ ]:
## Completely EMPTY VERSION
## TODO: Write a function called plot_top_grossing_movies that takes a data frame.
## TODO: Inside, use mutate() and parse_number() to clean the lifetime_gross column.
## TODO: Then, create a ggplot with geom_col() to visualize the top 10 movies.

------------------------------------------------------------------------

#### **Step 3: Plot the Scraped Data**

Finally, let’s use our new function with the `top_movies_df` data frame
we created in Step 1 to generate the plot.

**Filled Version (Answer)**

In [ ]:
# Use the function to plot the scraped movie data
plot_top_grossing_movies(top_movies_df)

**Fill-in-the-Blank Version**

In [ ]:
## EMPTY VERSION
## Use the function to plot the scraped movie data
# plot_top_grossing_movies(____)

**Scratch Version Instructions:** Now, simply call the
`plot_top_grossing_movies()` function that you just created. Pass the
`top_movies_df` data frame—which you scraped in the first step of this
project—as the single argument to the function to generate your final
visualization.

In [ ]:
## Completely EMPTY VERSION
## TODO: Call your plot_top_grossing_movies function with the top_movies_df data frame.

## Part 3. Reflection

1.  Compare the data cleaning required for the API response in Project 1
    (Geocoding) versus the scraped table in Project 2 (Box Office Mojo).
    Which required more work to prepare for plotting, and what does this
    tell you about the trade-offs between using APIs and web scraping?
2.  The Box Office Mojo scraper works today, but the website could be
    redesigned at any time. What specific part of our scraping code
    (`pluck(1)`) is most likely to break if the website’s layout
    changes, and what would be your first step to debug it?
3.  Our `get_city_coords` function is useful, but it assumes the API key
    is stored in a specific way (`Sys.getenv('OPENWEATHER_API_KEY')`).
    What argument could you add to this function to make it more
    flexible and easier for another person to use with their own key?
4.  Think about the two datasets we created: one with city coordinates
    and one with movie revenues. What is a new, interesting question you
    could potentially answer by finding a way to **join** these two
    datasets (perhaps with a third dataset)? What would be the biggest
    challenge in that process?